In [2]:
import pandas as pd
import numpy as np


In [4]:
df = pd.read_csv('diabetes.csv')

In [ ]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [5]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [6]:
x = df.drop(columns=['Outcome'])

y = df['Outcome']

In [7]:
from sklearn.model_selection import train_test_split

x_train , x_test , y_train , y_test = train_test_split(x , y , test_size = 0.2  , random_state = 2)

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [9]:
x_train.shape

(614, 8)

In [10]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [11]:
model = Sequential()

model.add(Dense(32 , activation = 'relu' , input_dim=8))
model.add(Dense(1 , activation = 'sigmoid'))



model.compile(optimizer = 'Adam' , loss='binary_crossentropy' ,metrics =['accuracy'] )


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
model.fit(x_train_scaled, y_train, batch_size=32, epochs=100, verbose=1 , validation_data=(x_test , y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.5166 - loss: 0.6892 - val_accuracy: 0.3506 - val_loss: 9.4274
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6787 - loss: 0.6370 - val_accuracy: 0.3052 - val_loss: 15.6856
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7106 - loss: 0.6071 - val_accuracy: 0.2922 - val_loss: 21.9292
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7281 - loss: 0.5676 - val_accuracy: 0.2922 - val_loss: 29.4456
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7425 - loss: 0.5520 - val_accuracy: 0.2922 - val_loss: 35.2254
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7289 - loss: 0.5339 - val_accuracy: 0.2922 - val_loss: 39.7889
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7714 - loss: 0.5111 - val_accuracy: 0.2922 - val_loss: 43.7053
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7705 - loss: 0.4949 - val_accura

In [13]:
#1.how to select appropriate optimiser
#2. no of nodes in a layer
#3. how to selct number of hidden layers
#4.all in all one model


In [14]:
pip install -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 9.6 MB/s eta 0:00:00


In [15]:
import kerastuner as kt

/tmp/ipython-input-1654478174.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [16]:
def build_model(hp):
  model = Sequential();
  model.add(Dense(32 , activation = 'relu' , input_dim=8))
  model.add(Dense(1 , activation = 'sigmoid'))
  optimizer=hp.Choice('optimizer',values=['adam' ,'sgd' , 'rmsprop' , 'adadelta'])
  model.compile(optimizer= optimizer , loss='binary_crossentropy' , metrics=['accuracy'])

  return model

In [17]:
tuner=kt.RandomSearch(build_model,
                      objective='val_accuracy',
                      max_trials=5)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [18]:
tuner.search(x_train_scaled , y_train , epochs = 5 , validation_data = (x_test , y_test))

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.3636363744735718

Best val_accuracy So Far: 0.7077922224998474
Total elapsed time: 00h 00m 29s


In [19]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adadelta'}

In [29]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [30]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
model.fit(x_train_scaled , y_train , batch_size=32 , epochs = 100 , initial_epoch = 6, validation_data=(x_test , y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7992 - loss: 0.4285 - val_accuracy: 0.2922 - val_loss: 72.0003
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7909 - loss: 0.4013 - val_accuracy: 0.2922 - val_loss: 70.8818
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7858 - loss: 0.3982 - val_accuracy: 0.2922 - val_loss: 72.0562
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8007 - loss: 0.4032 - val_accuracy: 0.2922 - val_loss: 67.9824
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8117 - loss: 0.3800 - val_accuracy: 0.2922 - val_loss: 69.2955
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7765 - loss: 0.4243 - val_accuracy: 0.2922 - val_loss: 70.2924
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8021 - loss: 0.4031 - val_accuracy: 0.2922 - val_loss: 70.8209
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8094 - loss: 0.3862 - val_accur

In [49]:
def build_model(hp):
  model=Sequential()
  units=hp.Int('units' , min_value = 8, max_value = 128)

  model.add(Dense(units=units , activation='relu' ,input_dim=8))
  model.add(Dense(1 , activation='sigmoid'))

  model.compile(optimizer='adam',loss='binary_crossentropy' , metrics=['accuracy'])

  return model

In [50]:
tuner = kt.RandomSearch(build_model ,
                        objective = 'val_accuracy',
                        max_trials = 5,
                        directory='mydir')

Reloading Tuner from mydir/untitled_project/tuner0.json


In [51]:
tuner.search(x_train_scaled  , y_train, epochs=5 , validation_data=(x_test , y_test))

In [52]:
tuner.get_best_hyperparameters()[0].values

{'units': 120}

In [53]:
model = tuner.get_best_models(num_models =1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [54]:
model.fit(x_train_scaled, y_train ,batch_size=32, epochs=100,initial_epoch=6 , validation_data=(x_test , y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6281 - loss: 0.6665 - val_accuracy: 0.2987 - val_loss: 5.6652
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7204 - loss: 0.5759 - val_accuracy: 0.2922 - val_loss: 19.6819
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7665 - loss: 0.5340 - val_accuracy: 0.2922 - val_loss: 27.3711
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7793 - loss: 0.5141 - val_accuracy: 0.2922 - val_loss: 35.6004
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7944 - loss: 0.4720 - val_accuracy: 0.2922 - val_loss: 42.3537
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7692 - loss: 0.4708 - val_accuracy: 0.2922 - val_loss: 40.9311
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7663 - loss: 0.5005 - val_accuracy: 0.2922 - val_loss: 47.0662
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7799 - loss: 0.4599 - val_accuracy

In [74]:
def build_model(hp):
    model = Sequential()

    num_layers = hp.Int('num_layers', min_value=1, max_value=10)

    # first hidden layer
    model.add(Dense(120, activation='relu', input_dim=8))

    # remaining hidden layers
    for i in range(num_layers - 1):
        model.add(Dense(120, activation='relu'))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model



In [80]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    directory='mydir',
    project_name='num_layers_only_v2'
)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [81]:
tuner.search(x_train_scaled , y_train, epochs = 5 , validation_data = (x_test , y_test) , verbose=1)

Trial 9 Complete [00h 00m 04s]
val_accuracy: 0.29870128631591797

Best val_accuracy So Far: 0.29870128631591797
Total elapsed time: 00h 00m 44s


In [82]:
tuner.get_best_hyperparameters()[0].values


{'num_layers': 9}

In [90]:
#going to calculate all the parameters in one go

def build_model(hp):
    model = Sequential()

    num_layers = hp.Int('num_layers', min_value=1, max_value=10)

    for i in range(num_layers):
        units = hp.Int(f'units_{i}', min_value=8, max_value=128, step=8)
        activation = hp.Choice(f'activation_{i}', ['relu', 'tanh', 'sigmoid'])

        if i == 0:
            model.add(Dense(units, activation=activation, input_dim=8))
        else:
            model.add(Dense(units, activation=activation))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=hp.Choice('optimizer', ['rmsprop', 'sgd', 'nadam', 'adadelta', 'adam']),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model


In [91]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=3,
                        directory='mydir',
                        project_name = 'final')

In [92]:
tuner.search(x_train_scaled , y_train, epochs = 5 , validation_data = (x_test , y_test) , verbose=1)

Trial 3 Complete [00h 00m 04s]
val_accuracy: 0.7077922224998474

Best val_accuracy So Far: 0.7077922224998474
Total elapsed time: 00h 00m 22s


In [93]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 9,
 'units_0': 104,
 'activation_0': 'sigmoid',
 'optimizer': 'adadelta',
 'units_1': 72,
 'activation_1': 'relu',
 'units_2': 32,
 'activation_2': 'sigmoid',
 'units_3': 16,
 'activation_3': 'relu',
 'units_4': 56,
 'activation_4': 'sigmoid',
 'units_5': 80,
 'activation_5': 'tanh',
 'units_6': 32,
 'activation_6': 'sigmoid',
 'units_7': 8,
 'activation_7': 'relu',
 'units_8': 8,
 'activation_8': 'relu'}

In [96]:
model = tuner.get_best_models(num_models=1)[0]


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adadelta', because it has 2 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [99]:
model.fit(x_train , y_train , epochs = 200 , initial_epoch=6 , validation_data=(x_test , y_test))

Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - accuracy: 0.6434 - loss: 0.6548 - val_accuracy: 0.7078 - val_loss: 0.6279
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6057 - loss: 0.6706 - val_accuracy: 0.7078 - val_loss: 0.6278
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6115 - loss: 0.6682 - val_accuracy: 0.7078 - val_loss: 0.6278
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6375 - loss: 0.6573 - val_accuracy: 0.7078 - val_loss: 0.6277
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6393 - loss: 0.6565 - val_accuracy: 0.7078 - val_loss: 0.6277
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6444 - loss: 0.6543 - val_accuracy: 0.7078 - val_loss: 0.6276
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6547 - loss: 0.6500 - val_accuracy: 0.7078 - val_loss: 0.6276
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6217 - loss: 0.6638 - val_accuracy: 